<h1><center>  定向利率交易 -  Nucleus API实例</center></h1>


<h1><center>  所有权及保密条款属SumUp Analytics所有</center></h1>


<h1><center>  免责声明和服务条款可通过 www.sumup.ai 获取</center></h1>

 


## 目标: 
-	利用中央银行出版物等的一系列宏观经济情绪，制定市场反弹/抛售指标


## 数据:
-	所有文件均由中国人民银行以普通话发布，不包括正式研究
 - 	演讲
 - 	新闻稿
 - 	非正式出版物

    **Nucleus Datafeed可调用出主要中央银行的所有内容**


## Nucleus APIs:
-	数据集创建 API
 - 	*api_instance.post_upload_file(file, dataset)*
 - 	*nucleus_helper.import_files(api_instance, dataset, file_iters, processes=1)*

        nucleus_helper.import_files利用api_instance.post_upload_file并行执行来加速数据集的创建


-	主题建模 API
 - 	*api_instance.post_topic_api(payload)*


-	主题情绪 API
 - 	*api_instance.post_topic_sentiment_api(payload)*


-	DocInfo API
 - 	*api_instance.post_doc_info(payload)*


-	DatasetInfo API
 - 	*api_instance.post_dataset_info(payload)*


## 方法:

### 1.	数据集准备
-	创建一个Nucleus数据集，其中包含所选历史时期内的所有相关文档

    

In [ ]:
print('---- Append all files from local folder to dataset in parallel ----')
dataset = 'sumup/central_banks_chinese'# Nucleus中自带数据馈送。
metadata_selection = {'bank': 'people_bank_of_china', 'document_category': ('speech', 'press release', 'publication')}

-	对于给定日期，保留在选定的回顾期间发布的文档的子集

**这可以直接在执行内容分析的API中完成，如下所示**



### 2.	情感分析
-	在该日期的文档子集上确定并提取给定时间点的关键主题


-	衡量每个主题的情绪，并将每个主题的情绪汇总到一个PBOC级别的情绪中


-	接下来，我们将讨论如何通过利用用户可用的不同参数来优化情绪分析



In [ ]:
import csv
import json
import nucleus_api.api.nucleus_api as nucleus_helper
import nucleus_api
from nucleus_api.rest import ApiException

configuration = nucleus_api.Configuration()
configuration.host = 'UPDATE-WITH-API-SERVER-HOSTNAME'
configuration.api_key['x-api-key'] = 'UPDATE-WITH-API-KEY'

# 创建API实例
api_instance = nucleus_api.NucleusApi(nucleus_api.ApiClient(configuration))

In [ ]:
import numpy as np

print('---------------- Get topic sentiment ------------------------')

payload = nucleus_api.TopicSentimentModel(dataset='sumup/central_banks_chinese',           
                                query='earnings OR ESG',                   
                                num_topics=8,
                                num_keywords=8,
                                metadata_selection=metadata_selection,
                                period_start="2018-11-01 00:00:00",
                                period_end="2019-01-01 00:00:00")
try:
    api_response = api_instance.post_topic_sentiment_api(payload)
    api_ok = True
except ApiException as e:
    api_error = json.loads(e.body)
    print('ERROR:', api_error['message'])
    api_ok = False

if api_ok:       
    for i, res in enumerate(api_response.result):
        print('Topic', i, ' keywords: ', res.keywords)    
        print('    Sentiment:', res.sentiment)
        print('---------------')

    # 汇总所有主题的情绪
    PBOC_sent = np.dot(api_response.result.sentiment, api_response.result.strength)

-	Repeat the above tasks for each date in the historical period to get the complete history of your sentiment indicator

In [ ]:
import datetime
import numpy as np

print('--------------- Retrieve the time range of the dataset -------------')

payload = nucleus_api.DatasetInfo(dataset='sumup/central_banks_chinese', query='')
api_response = api_instance.post_dataset_info(payload)

first_date = datetime.datetime.fromtimestamp(float(api_response.result.time_range[0]))
last_date = datetime.datetime.fromtimestamp(float(api_response.result.time_range[1]))
delta = last_date – first_date

# 遍历时间和每个日期，计算PBOC的情绪指标
T = 90 # The look-back period in days

PBOC_sentiments = []
for i in range(delta.days):  
    if i == 1:
        end_date = first_date + datetime.timedelta(days=T)
 
    # 用于T天回溯期的第一个和最后一个日期
    start_date = end_date - datetime.timedelta(days=T)
    start_date_str = start_date.strftime("%Y-%m-%d 00:00:00")

    # 我们想要一个每日指标
    end_date = end_date + datetime.timedelta(days=1) 
    end_date_str = end_date.strftime("%Y-%m-%d 00:00:00")

    payload = nucleus_api.TopicSentimentModel(dataset="sumup/central_banks_chinese",        
                                            query='',                   
                                            num_topics=8,
                                            num_keywords=8,
                                            metadata_selection=metadata_selection,
                                            period_start= start_date_str,
                                            period_end= end_date_str)
    try:
        api_response = api_instance.post_topic_sentiment_api(payload)
        api_ok = True
    except ApiException as e:
        api_error = json.loads(e.body)
        print('ERROR:', api_error['message'])
        api_ok = False

    if api_ok:   
        # 汇总所有主题的情绪
        PBOC_sentiments.append(np.dot(api_response.result.sentiment, api_response.result.strength))

### 3.	结果解释
-	绘制中国人民银行对政府收益率，信贷利差指数，股票指数的情绪时间序列



### 4.	微调

#### a.	量身定制主题

-   通过使用Topic Sentiment API中的`custom stop_words`参数排除不相关或次要主题以定制您的情绪度量

-	提取文档子集的关键主题并打印其关键字



In [ ]:
print('------------- Get list of topics from dataset --------------')

payload = nucleus_api.Topics(dataset='sumup/central_banks_chinese',                         
                            query='',                       
                            num_topics=8, 
                            num_keywords=8,
                            metadata_selection=metadata_selection,
                            period_start="2018-11-01 00:00:00",
                            period_end="2019-01-01 00:00:00")
try:
    api_response = api_instance.post_topic_api(payload)        
    api_ok = True
except ApiException as e:
    api_error = json.loads(e.body)
    print('ERROR:', api_error['message'])
    api_ok = False

if api_ok:       
    for i, res in enumerate(api_response.result.topics):
        print('Topic', i, ' keywords: ', res.keywords)    
        print('---------------')

然后，您可以通过创建custom_stop_words变量来定制情绪分析。 例如，如下所示初始化变量，并将其传递到第2节主代码的有效负载(payload)中：

In [ ]:
custom_stop_words = ["conference","government"] # str | 停用词列表。 （可选）

#### b.	将情绪分析集中在某些主题上
如果您决定关注情绪分析，例如政策和宏观经济主体，只需将第2节主要代码中的查询变量替换为： 

In [ ]:
query = '(inflation OR growth OR unemployment OR stability OR regulation)' # str | 全文查询，使用mysql MATCH boolean query 格式。 示例：“（word1 OR word2）AND（word3 OR word4）”（可选）

#### c.	了解文档类型，回顾期，提取的主题数量对结果的影响
**num_topics**: 您可以通过更改有效负载（payload）中的变量`num_topics`来计算公司对不同广度主题的共识。
- 在建立情绪指标时，较大的值将包含更广泛的主题/内容，而较小的值将提供具有主要主题的简明度量。 
- 如果`num_topics`太大，一些非常边缘的话题可能会在测量情绪时带来很多噪音。


**T**: 您可以通过更改第2节主代码中的变量`T`（lookback）来计算具有不同传播速度的情绪指标。
- 较大的价值将提供缓慢变化的情绪衡量标准，而较小的价值将导致非常敏感的情绪衡量标准。
- 如果T太小，将只使用很少的文件，这可能会在测量情绪时产生很多噪音。
- 如果T太大，由于平滑效应，情绪指标不会反映信息。

**Document types**: 与捕获整个数据集相比，您可以调查情绪指标如何随不同类型的文档（例如语音，新闻稿或出版物）而变化。 在整个语料库的子集上重新运行第2节的主要代码。 创建一个变量`metadata_selection`并将其传递给有效负载：

In [ ]:
metadata_selection = {"document_category": "speech"}   # str | json object of {\"metadata_field\":[\"selected_values\"]} (optional)

### 5.	下一步
-	为每个（美联储，欧洲中央银行，英格兰银行，加拿大银行，日本银行，澳大利亚皇家银行，德国央行）重复上述任务
 - 这为每个国家/地区提供了一个指标，可以为每个国家/地区提供独立信号
 - 您还可以根据可以提供安全选择信号的情绪度量对每个国家进行排名


-	在情绪指标的时间序列与政府收益率，信用利差指数，股票指数之间进行相关性分析
 - 可以研究价格影响的几个时间范围：1天，7天，几周，甚至更持久的影响
 - 可以研究几个时间滞后的价格影响：第二天，市场开始调整之前的2至3天差距，一周，甚至更长的差距，然后市场开始纳入来自中央银行情绪的信息
 - 哪种标的资产似乎反响最强？ 在一项资产中，哪一类的回应更多：实物证券或期货？


-	探索指标的简单转换 
 - 如果您有独立的每个国家/地区指标，则可以使用时间序列分数重新调整和平滑这些指标

            Score(t) = ( Indicator(t) – Average(Indicator, [t – N  ; t]) ) / Std(Indicator, [t – N  ; t])

 - 如果您根据每个情绪指标对国家/地区进行排名，则可以使用横截面得分重新调整并平滑这些排名

            Score(Bank i) = ( Indicator( Bank i) – Average(Indicator, [Banks]) ) / Std(Indicator, [Banks])




In [ ]:
# 创建回归对象

# 训练模型
# 情绪指标的变化是x
# 价格变动或任何其他可交易资产是y

# 1. 预测费率的方向
regr.fit(change in SentimentIndicator(t- p), sign(change in rate )(t)) # 指标与市场反应之间可能存在滞后= p。 去测试。

# 2.预测移动的方向和大小
regr.fit(change in SentimentIndicator(t- p), (change in rates)(t))

fitted_score = regr.score(change in SentimentIndicator, change in rates)

# 使用该doc对主题+拟合的曝光，预测最新发布的doc的股票回报
y_predicted = regr.fit(x, y).predict(x.reshape(1,-1))

版权(c) 2019年 SumUp Analytics 公司 版权所有。 

通知：所有信息均属于SumUp Analytics Inc公司及其供应商的财产。 本合同所包含的知识产权和技术概念属于SumUp Analytics Inc.及其供应商的专利，可由美国和外国专利、在工艺中的专利以及受贸易秘密或版权法保护的专利涵盖。 

除非得到SumUp Analytics公司的事先书面批准，否则严禁传播此类信息或复制此材料。 